In [1]:
import os
import pickle

import pandas as pd
import numpy as np

First need to understand how data is structured.

In [ ]:
with open("data/01-raw/0/t_1000000.c", "r") as file:
        data = file.read()

In [ ]:
print(data)

In [4]:
def parse(data):
    parsed_data = [x.split()[:2] for x in data.split("\n")[2:-1]]
    parsed_data = np.asarray(parsed_data, dtype=float)
    return parsed_data

In [70]:
parse(data).shape

(1000, 2)

In [71]:
count = 0
for file_name in os.listdir("data/01-raw/0/"):
    if file_name.startswith("t_"):
        count += 1

count

21

There are 26 folders corresponding to different values of n_p as described in  
key. Each folder contains 21 different samples for this value of n_p, each  
sample consists of 1000 2-dimensional coordinates.

We will therefore store the data in a numpy array with shape (26, 21, 1000, 2).  
To keep track of things, we'll also store a pandas frame containing 21 x 25  
entries, tracking the samples for different values of Phi and n_p. This pandas  
frame will be used as a template for later investigation.

In [72]:
for folder_name in sorted(os.listdir("data/01-raw/")):
    print(folder_name)

.DS_Store
0
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
3
4
5
6
7
8
9
README.md
key.txt


Need to pad the names with zeros on the left so can iterate over in order. 

In [2]:
raw_data_path = "data/01-raw/"

for folder_name in os.listdir(raw_data_path):
    if folder_name.isdigit():
        new_folder_name = folder_name.zfill(2)
        os.rename(raw_data_path + folder_name, raw_data_path + new_folder_name)

In [5]:
raw_data = [] 
for folder_name in sorted(os.listdir(raw_data_path)):
    # Ignore folders which aren't a digit
    if not folder_name.isdigit():
        continue
    
    samples = []

    # Access samples from folder in time order.
    is_sample = lambda x : x.startswith("t_")
    for file_name in sorted(
        filter(is_sample, os.listdir(raw_data_path + folder_name)),
        key=lambda x: int(x[2:-2])
    ):
        with open(raw_data_path + folder_name + "/" + file_name, "r") as file:
            data = file.read()
            data = parse(data)
        
        samples.append(data) 
    
    raw_data.append(samples) 

In [6]:
raw_data = np.array(raw_data)

In [7]:
raw_data.shape

(26, 21, 1000, 2)

In [97]:
with open(raw_data_path + "key.txt", "r") as file:
    print(file.read())

id n_b Phi
0 0.5 0.25
1 0.6 0.25
2 0.7 0.25
...
...
...
25 3.0 0.25


In [8]:
n_b = np.arange(0.5, 3.1, 0.1)
Phi = [0.25]

In [9]:
t = []
for file_name in filter(is_sample, os.listdir(raw_data_path + "00/")):
    t.append(int(file_name[2:-2]))
t.sort()

In [10]:
key = pd.DataFrame(index=t, columns=pd.MultiIndex.from_product([Phi, n_b]),
                   dtype=float)

Save data.

In [11]:
with open("data/02-preprocessed/data.pickle", "wb") as file:
    pickle.dump(raw_data, file)

key.to_pickle("data/02-preprocessed/key.pickle")